<a href="https://colab.research.google.com/github/anshupandey/Machine_Learning_Training/blob/master/fidelity/RVL_CDIP_code3_classification_with_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow-text

     |████████████████████████████████| 4.3 MB 8.2 MB/s 
     |████████████████████████████████| 454.4 MB 10 kB/s 
     |████████████████████████████████| 462 kB 38.1 MB/s 
     |████████████████████████████████| 4.0 MB 41.8 MB/s 
     |████████████████████████████████| 1.2 MB 53.1 MB/s 
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.39.0
    Uninstalling grpcio-1.39.0:
      Successfully uninstalled grpcio-1.39.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.6.0
    Uninstalling tensorflow-2.6.0:
      Successfully uninstalled tensorflow-2.6.0


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import models,layers
import tensorflow_text as text

In [ ]:
text_input = layers.Input(shape=(), dtype=tf.string)
preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder_inputs = preprocessor(text_input)
encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/2",trainable=False)
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"]# [batch_size, 512]. - sentence vectorization
sequence_output = outputs["sequence_output"]# [batch_size, seq_length, 512]. - word vectoriation

In [ ]:
embedding_model = tf.keras.Model(text_input, pooled_output)
sentences = tf.constant(["Hello, how are you doing?"])
print(embedding_model(sentences).shape)

(1, 512)


In [ ]:
embedding_model2 = tf.keras.Model(text_input, sequence_output)
sentences = tf.constant(["Hello, how are you doing?"])
print(embedding_model2(sentences).shape)

(1, 128, 512)


## Modelling the network

In [ ]:
# create a function which accepts a text doc and return corresponding vector
def doc2vec(doc):
  vector = embedding_model(tf.squeeze(tf.cast(doc,tf.string),axis=1))
  return vector

In [ ]:
input_layer = layers.Input(shape=(1,),dtype='string')
# adding embedding layer - BERT
emb = layers.Lambda(doc2vec,output_shape=(512,))(input_layer)
dense1 = layers.Dense(256,activation='relu')(emb)
op_layer = layers.Dense(5,activation='softmax')(dense1)

model = models.Model(inputs=input_layer,outputs=op_layer)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

The following Variables were used a Lambda layer's call (lambda), but
are not present in its tracked objects:
  <tf.Variable 'word_embeddings/embeddings:0' shape=(30522, 512) dtype=float32>
  <tf.Variable 'position_embedding/embeddings:0' shape=(512, 512) dtype=float32>
  <tf.Variable 'type_embeddings/embeddings:0' shape=(2, 512) dtype=float32>
  <tf.Variable 'embeddings/layer_norm/gamma:0' shape=(512,) dtype=float32>
  <tf.Variable 'embeddings/layer_norm/beta:0' shape=(512,) dtype=float32>
  <tf.Variable 'transformer/layer_0/self_attention/query/kernel:0' shape=(512, 8, 64) dtype=float32>
  <tf.Variable 'transformer/layer_0/self_attention/query/bias:0' shape=(8, 64) dtype=float32>
  <tf.Variable 'transformer/layer_0/self_attention/key/kernel:0' shape=(512, 8, 64) dtype=float32>
  <tf.Variable 'transformer/layer_0/self_attention/key/bias:0' shape=(8, 64) dtype=float32>
  <tf.Variable 'transformer/layer_0/self_attention/value/kernel:0' shape=(512, 8, 64) dtype=float32>
  <tf.Variable 't

The following Variables were used a Lambda layer's call (lambda), but
are not present in its tracked objects:
  <tf.Variable 'word_embeddings/embeddings:0' shape=(30522, 512) dtype=float32>
  <tf.Variable 'position_embedding/embeddings:0' shape=(512, 512) dtype=float32>
  <tf.Variable 'type_embeddings/embeddings:0' shape=(2, 512) dtype=float32>
  <tf.Variable 'embeddings/layer_norm/gamma:0' shape=(512,) dtype=float32>
  <tf.Variable 'embeddings/layer_norm/beta:0' shape=(512,) dtype=float32>
  <tf.Variable 'transformer/layer_0/self_attention/query/kernel:0' shape=(512, 8, 64) dtype=float32>
  <tf.Variable 'transformer/layer_0/self_attention/query/bias:0' shape=(8, 64) dtype=float32>
  <tf.Variable 'transformer/layer_0/self_attention/key/kernel:0' shape=(512, 8, 64) dtype=float32>
  <tf.Variable 'transformer/layer_0/self_attention/key/bias:0' shape=(8, 64) dtype=float32>
  <tf.Variable 'transformer/layer_0/self_attention/value/kernel:0' shape=(512, 8, 64) dtype=float32>
  <tf.Variable 't

In [ ]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
lambda (Lambda)              (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 1285      
Total params: 132,613
Trainable params: 132,613
Non-trainable params: 0
_________________________________________________________________


## Data Preparation

In [ ]:
!wget -q https://www.dropbox.com/s/p3tpkhk53ypb2cn/final_data.zip

In [ ]:
!unzip final_data.zip > /dev/null; echo " done."

 done.


In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:
mainfolder = "/content/final_data/textdata/"
cnames = sorted(os.listdir(mainfolder))
cnames

['budget', 'form', 'handwritten', 'invoice', 'resume']

In [ ]:
x = []
y = []
for folder in cnames:
  files = os.listdir(mainfolder+folder)
  for file in files:
    path = mainfolder + folder + "/" + file
    with open(path,'r',encoding='cp1252') as file:
      x.append(file.read())
    y.append(cnames.index(folder))

In [ ]:
len(x),len(y)

(4628, 4628)

In [ ]:
x = np.array(x).reshape(-1,1)
from tensorflow.keras.utils import to_categorical
y = to_categorical(y)
print(x.shape,y.shape)

(4628, 1) (4628, 5)


In [ ]:
model.fit(x,y,epochs=4,batch_size=32,validation_split=0.2)

Epoch 1/4
116/116 [==============================] - 409s 3s/step - loss: 0.9804 - accuracy: 0.6099 - val_loss: 0.8570 - val_accuracy: 0.6523
Epoch 2/4
116/116 [==============================] - 392s 3s/step - loss: 0.8034 - accuracy: 0.6910 - val_loss: 0.7641 - val_accuracy: 0.7009
Epoch 3/4
 86/116 [=====================>........] - ETA: 1:20 - loss: 0.7771 - accuracy: 0.6962